폐배터리 최신순 뉴스 기사 

In [28]:
import os
import sys
import urllib.request
import datetime
import time
import json

#클라이언트 아이디, 비번 입력
client_id = "zhMWdi3bpEWm9qTBApcn"
client_secret = "3cb8cCFSPA"

# url 받고 요청하는 함수
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
    
    
 # open api 접속하고, url 주소 생성(최신순/관련도순)하는 함수
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search" #open API 접속
    node = "/%s.json" % node #파일 받을 형식
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display) #최신순 기사 파라미터 받기
    
    #sort='sim' 관련도순 기사 받는 경우 sort 설정

    url = base + node + parameters 
    responseDecode = getRequestUrl(url)  

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)
    
    
    
# 받고싶은 정보를 변수값에 저장
def getPostData(post, jsonResult, cnt):
    title = post['title'] #뉴스 제목
    description = post['description'] #메인 내용
    org_link = post['originallink'] #뉴스 페이지 원본(오리지널) 링크
    link = post['link'] #뉴스 페이지 링크

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt': cnt, 'title': title, 'description': description,
                       'org_link': org_link, 'link': link, 'pDate': pDate})
    return


# 크롤링 시작/ json 파일로 저장
def main():
    node = 'news'  # 크롤링 할 대상
    srcText = '폐배터리'
    cnt = 0
    jsonResult = [] #모든 데이터를 받는 리스트

    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # url 생성 함수 활용
    total = jsonResponse['total']

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)): #검색 결과 없으면 크롤링 안함 
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)  # 변수값 저장 함수 활용

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)  

    print('전체 검색 : %d 건' % total)

    with open('%s_naver_%s_1.json' % (srcText, node), 'w', encoding='utf8') as outfile: # 검색 결과 json 파일로 저장 
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)

        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" % (cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))


if __name__ == '__main__':
    main()

   


[2023-05-16 10:56:07.917849] Url Request Success
[2023-05-16 10:56:08.085520] Url Request Success
[2023-05-16 10:56:08.286442] Url Request Success
[2023-05-16 10:56:08.446113] Url Request Success
[2023-05-16 10:56:08.598695] Url Request Success
[2023-05-16 10:56:08.786389] Url Request Success
[2023-05-16 10:56:08.972908] Url Request Success
[2023-05-16 10:56:09.148316] Url Request Success
[2023-05-16 10:56:09.339865] Url Request Success
[2023-05-16 10:56:09.546230] Url Request Success
HTTP Error 400: Bad Request
[2023-05-16 10:56:09.662450] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%ED%8F%90%EB%B0%B0%ED%84%B0%EB%A6%AC&start=1001&display=100
전체 검색 : 25276 건
가져온 데이터 : 1000 건
폐배터리_naver_news.json SAVED


In [29]:
import json

#폐배터리 json 파일 열기
with open('폐배터리_naver_news_1.json', encoding='utf-8') as file:
    datas=json.load(file)
    datas=datas[:200] #200개만 추출
    #json_test=datas['title']
    #print(datas)
    
#for i in datas:
#    print(i)
    
#기사 제목, 기사 내용 전처리
# <b>, </b>, ◆, ▲, ◇, &apos;, &quot;, R&amp;D, /

import re

for i in datas:  #R&amp;D 
    
    title_final=re.sub('<b>|</b>|◆|▲|◇|&apos;|&quot;|R&amp;D|/| R&amp;D'   ,'',i['title'])
    i['title']=title_final
    
    description_final=re.sub('<b>|</b>|◆|▲|◇|&apos;|&quot;|/R&amp;D|/| R&amp;D'   ,'',i['description'])
    i['description']=description_final
    
    print(i['description'])
    print(i['title'])


# '기사번호','기사제목','기사내용','날짜','링크','원본링크' 칼럼 생성
# 'cnt', 'description', 'link', 'org_link', 'pDate', 'title'

#number=[] #cnt
title=[] #title
description=[] #description
date=[] #pDate
link=[] #link
original_link=[] #org_link

for i in datas:
    title.append(i['title'])

#for i in datas:
#    number.append(i['cnt'])
    
for i in datas:
    description.append(i['description'])

for i in datas:
    date.append(i['pDate'])
for i in datas:
    link.append(i['link'])
    
for i in datas:
     original_link.append(i['org_link'])
    
    
import pandas as pd

#최신순 df 생성
df_1=pd.DataFrame(list(zip(title, description, date, link, original_link)),
                columns=['기사제목','기사내용','날짜','링크','원본링크'])       
 
print(df_1)

#엑셀파일 생성
df_1.to_excel('폐배터리df_1_3.xlsx',index=False)
#df_comment.to_excel('df_comment3.xlsx')

그러면서 환경과학기술원은 SK이노베이션 경영의 시초로, 지난 40년 간 SK이노베이션 계열의 비약적인 성장을 이끌어왔다며 향후 40년도 지속적인 혁신을 통해 암모니아, 폐배터리 재활용(BMR) 등 미래에너지... 
김준 SK이노 부회장 미래 그린 에너지 산업 선도할 주역 돼 달라
김 부회장은 기술원은 SK이노베이션 경영의 시초로, 지난 40년간 SK이노베이션 계열의 비약적인 성장을 이끌어왔다며 앞으로 40년도 지속적인 혁신을 통해 암모니아, 폐배터리 재활용(BMR) 등 미래에너지 사업의... 
SK이노 경영 40년…김준 부회장, 환경과학기술원 찾아 ‘미래 그린 에너지...
그러면서 환경과학기술원은 SK이노베이션 경영의 시초로, 지난 40년 간 SK이노베이션 계열의 비약적인 성장을 이끌어왔다며 향후 40년도 지속적인 혁신을 통해 암모니아, 폐배터리 재활용(BMRㆍBattery Metal... 
김준 부회장 미래 그린 에너지 산업 선도 주역 돼 달라
그러면서 환경과학기술원은 SK이노베이션 경영의 시초로, 지난 40년 간 SK이노베이션 계열의 비약적인 성장을 이끌어왔다며 향후 40년도 지속적인 혁신을 통해 암모니아, 폐배터리 재활용 등 미래에너지... 
김준, 환경과학기술원서 SK이노 미래 그린 산업 선도 주역 당부
김 부회장 또한 환경과학기술원은 SK이노베이션 경영의 시초로, 지난 40년 간 SK이노베이션 계열의 비약적인 성장을 이끌어왔다며 향후 40년도 지속적인 혁신을 통해 암모니아, 폐배터리 재활용... 
김준 SK이노베이션 부회장 로 친환경 사업 선도
전북도는 전북대, 한국전기안전공사, 전북테크노파크(TP)와 업무협약을 맺고 이차전지 생산에서 폐배터리 재활용까지 이르는 전주기 안전성 확보에 힘을 모으기로 했다고 16일 밝혔다. 도는 이날 김관영 전라북도지사... 
전북도, 국내 최대 규모 이차전지 안전기술 개발·평가·인증센터 구축 운영
나아가 사업 포트폴리오까지 바이오, 재활용, 재생에너지 소재 등 저탄소 중심으로 전환을 가속화하고 있으며, 폐배터리 자원 선

폐배터리 관련도순 뉴스기사

In [30]:
# 네이버 기사 관련도순 불러오기


import os
import sys
import urllib.request
import datetime
import time
import json

#클라이언트 아이디, 비번 입력
client_id = "zhMWdi3bpEWm9qTBApcn"
client_secret = "3cb8cCFSPA"

# url 받고 요청하는 함수
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
    
    
 # open api 접속하고, url 주소 생성(최신순/관련도순)하는 함수
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search" #open API 접속
    node = "/%s.json" % node #파일 받을 형식
    parameters = "?query=%s&start=%s&display=%s&sort=%s" % (urllib.parse.quote(srcText), start, display,'sim') #기사 파라미터 받기 #관련도순('sim')으로 설정
    
    url = base + node + parameters
    responseDecode = getRequestUrl(url)  

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)
    
    
    
# 받고싶은 정보를 변수값에 저장하는 함수
def getPostData(post, jsonResult, cnt):
    title = post['title'] #뉴스 제목
    description = post['description'] #메인 내용
    org_link = post['originallink'] #뉴스 페이지 원본(오리지널) 링크
    link = post['link'] #뉴스 페이지 링크

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt': cnt, 'title': title, 'description': description,
                       'org_link': org_link, 'link': link, 'pDate': pDate})
    return


# 크롤링 시작/ json 파일로 저장
def main():
    node = 'news'  # 크롤링 할 대상
    srcText = '폐배터리'
    cnt = 0
    jsonResult = [] #모든 데이터를 받는 리스트

    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # url 생성 함수 활용
    total = jsonResponse['total']

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)): #검색 결과 없으면 크롤링 안함 
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)  # 변수값 저장 함수 활용

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)  

    print('전체 검색 : %d 건' % total)

    with open('%s_naver_%s_1.json' % (srcText, node), 'w', encoding='utf8') as outfile: # 검색 결과 json 파일로 저장 
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)

        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" % (cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))


if __name__ == '__main__':
    main()

[2023-05-16 10:56:13.199146] Url Request Success
[2023-05-16 10:56:13.366606] Url Request Success
[2023-05-16 10:56:13.555620] Url Request Success
[2023-05-16 10:56:13.734861] Url Request Success
[2023-05-16 10:56:13.900551] Url Request Success
[2023-05-16 10:56:14.116197] Url Request Success
[2023-05-16 10:56:14.288584] Url Request Success
[2023-05-16 10:56:14.480115] Url Request Success
[2023-05-16 10:56:14.664840] Url Request Success
[2023-05-16 10:56:14.866004] Url Request Success
HTTP Error 400: Bad Request
[2023-05-16 10:56:15.001250] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%ED%8F%90%EB%B0%B0%ED%84%B0%EB%A6%AC&start=1001&display=100&sort=sim
전체 검색 : 25276 건
가져온 데이터 : 1000 건
폐배터리_naver_news.json SAVED


In [31]:
import json

#폐배터리 json 파일 열기
with open('폐배터리_naver_news_1.json', encoding='utf-8') as file:
    datas=json.load(file)
    datas=datas[:200] #200개만 추출
    #json_test=datas['title']
    #print(datas)
    
#for i in datas:
#    print(i)
    
#기사 제목, 기사 내용 전처리
# <b>, </b>, ◆, ▲, ◇, &apos;, &quot;, R&amp;D, /

import re

for i in datas:  #R&amp;D 
    
    title_final=re.sub('<b>|</b>|◆|▲|◇|&apos;|&quot;|R&amp;D|/| R&amp;D'   ,'',i['title'])
    i['title']=title_final
    
    description_final=re.sub('<b>|</b>|◆|▲|◇|&apos;|&quot;|/R&amp;D|/| R&amp;D'   ,'',i['description'])
    i['description']=description_final
    
    print(i['description'])
    print(i['title'])


# '기사번호','기사제목','기사내용','날짜','링크','원본링크' 칼럼 생성
# 'cnt', 'description', 'link', 'org_link', 'pDate', 'title'

#number=[] #cnt
title=[] #title
description=[] #description
date=[] #pDate
link=[] #link
original_link=[] #org_link

for i in datas:
    title.append(i['title'])

#for i in datas:
#    number.append(i['cnt'])
    
for i in datas:
    description.append(i['description'])

for i in datas:
    date.append(i['pDate'])
for i in datas:
    link.append(i['link'])
    
for i in datas:
     original_link.append(i['org_link'])
    
    
import pandas as pd

#관련도순 df 생성
df_2=pd.DataFrame(list(zip(title, description, date, link, original_link)),
                columns=['기사제목','기사내용','날짜','링크','원본링크'])       
 
print(df_2)

#엑셀파일 생성
df_2.to_excel('폐배터리df_2_1.xlsx',index=False)
#df_comment.to_excel('df_comment3.xlsx')

정부가 전기차 폐배터리산업 활성화를 위해 규제 개선에 나섰다. 전기차 시대가 본격화되면서 폐배터리 증가가 예고된 만큼 기업들이 사업 확장에 어려움을 겪지 않도록 하기 위해서다. 16일 시장조사업체... 
폐배터리 쏟아진다… 정부, 규제 개선 통해 관련 사업 지원
환경부가 전기자동차 폐배터리 재활용을 활성화하기 위해 환경규제 개선에 나섰다. 유제철 환경부 차관은 15일 오후 전기차 폐배터리 재활용 기업인 성일하이텍을 방문, 재활용산업 현장 목소리를 듣고 지원방안을... 
환경부, 전기차 폐배터리 재활용 활성화…환경규제 개선 추진
사천 대진자원순환단지 폐배터리 순환 사업 계획을 15일 공개했다. 이 사업 시행사인 대진일반산업단지... 대진자원순환단지 폐배터리 재생 사업은 폐배터리에서 원료 금속을 추출해 재활용하는 리사이클링 존... 
SK에코플랜트, 사천 폐배터리 순환 사업 계획 공개
기사내용 요약 유제철 환경차관, 전기차 폐배터리 재활용기업 방문 폐기물관리법 하위법령 개정…개선기준... 이번 방문은 전기차 폐배터리 재활용 산업 육성·지원을 위해 현장 목소리를 청취하고 지원방안을 논의하는... 
환경부, 전기차 폐배터리 물량 확보 돕는다…규제 개선
SK에코플랜트의 자회사 싱가포르 테스(TES-AMM)가 포르투갈 최대 에너지기업과 전방위 협력을 통해 남유럽 폐배터리 재활용 시장 선점에 나선다. SK에코플랜트는 전기차 폐배터리 및 전기∙전자폐기물(E-waste)... 
SK에코플랜트 자회사 테스, 남유럽 폐배터리 재활용 시장 진출
이번 방문은 전기차 폐배터리 재활용 산업 육성·지원을 위해 현장 목소리를 청취하고 지원방안을 논의하는 차원에서 마련됐다. 그간 업계는 폐배터리와 공정스크랩 등 폐기물 보관 용량이 법적으로 한정돼 있어 원료... 
전기차 폐배터리 재활용산업 환경규제 개선
황덕현 기후환경전문기자 = 환경부는 유제철 차관이 15일 전북 군산 소재 전기차 폐배터리 재활용기업인... 환경부는 5월 중 폐기물관리법 하위법령(시행규칙) 개정 전에 개선기준을 사전에 시행

최신순, 관련도순 데이터프레임 합치고 중복 제거

In [32]:
#데이터 프레임 합치기
df_final= pd.concat([df_1,df_2])
print(df_final)
#df_final.to_excel('폐배터리df_final_1.xlsx',index=False)

                                             기사제목   
0            김준 SK이노 부회장 미래 그린 에너지 산업 선도할 주역 돼 달라  \
1    SK이노 경영 40년…김준 부회장, 환경과학기술원 찾아 ‘미래 그린 에너지...   
2                  김준 부회장 미래 그린 에너지 산업 선도 주역 돼 달라   
3             김준, 환경과학기술원서 SK이노 미래 그린 산업 선도 주역 당부   
4                      김준 SK이노베이션 부회장 로 친환경 사업 선도   
..                                            ...   
195                  충남도, 미래성장 동력 17개 사업 1781억 투자   
196          SK에코플랜트 1분기 영업익 480억 원···전년대비 4.1% ↑   
197                신한투자증권, 美 폐배터리 재활용 기업에 140억 투자   
198                      LG전자, 무선청소기 폐배터리 자원순환 앞장   
199                    LG전자, 청소기 폐배터리 회수로 자원순환 실천   

                                                  기사내용                   날짜   
0    그러면서 환경과학기술원은 SK이노베이션 경영의 시초로, 지난 40년 간 SK이노베이...  2023-05-16 10:54:00  \
1    김 부회장은 기술원은 SK이노베이션 경영의 시초로, 지난 40년간 SK이노베이션 계...  2023-05-16 10:49:00   
2    그러면서 환경과학기술원은 SK이노베이션 경영의 시초로, 지난 40년 간 SK이노베이...  2023-05-16 10:44:00   
3    그러면서 환경과학기술원은 SK이노베이션 경영의 시초로, 지난 40년 간 SK

In [35]:
#중복제거
df_final=df_final.drop_duplicates(subset=['기사제목'])
df_final=df_final.drop_duplicates(subset=['링크'])
df_final=df_final.drop_duplicates(subset=['원본링크'])
df_final=df_final.drop_duplicates(subset=['기사내용'])
print(df_final)

df_final.to_excel('폐배터리df_final_2.xlsx',index=False)


                                             기사제목   
0            김준 SK이노 부회장 미래 그린 에너지 산업 선도할 주역 돼 달라  \
1    SK이노 경영 40년…김준 부회장, 환경과학기술원 찾아 ‘미래 그린 에너지...   
2                  김준 부회장 미래 그린 에너지 산업 선도 주역 돼 달라   
3             김준, 환경과학기술원서 SK이노 미래 그린 산업 선도 주역 당부   
4                      김준 SK이노베이션 부회장 로 친환경 사업 선도   
..                                            ...   
190           새 배터리 할인…LG전자, 무선청소기 폐배터리 회수 캠페인 진행   
192           “폐배터리 반납시 새 배터리 할인“…LG전자, ‘배터리턴’ 진행   
193          신한증권, 美 폐배터리 재활용 ‘어센드 엘리먼츠’에 140억 투자   
195                  충남도, 미래성장 동력 17개 사업 1781억 투자   
199                    LG전자, 청소기 폐배터리 회수로 자원순환 실천   

                                                  기사내용                   날짜   
0    그러면서 환경과학기술원은 SK이노베이션 경영의 시초로, 지난 40년 간 SK이노베이...  2023-05-16 10:54:00  \
1    김 부회장은 기술원은 SK이노베이션 경영의 시초로, 지난 40년간 SK이노베이션 계...  2023-05-16 10:49:00   
2    그러면서 환경과학기술원은 SK이노베이션 경영의 시초로, 지난 40년 간 SK이노베이...  2023-05-16 10:44:00   
3    그러면서 환경과학기술원은 SK이노베이션 경영의 시초로, 지난 40년 간 SK